# Pointwise Reliability: Density, Distance - Euclidean
Uses distances between a new instance and all preprocessed training points to assess reliability

## Workflow:
- Calculate the Euclidean distances from the instance to all points in the training data
- Identify points in the training set that belong to the same class as the predicted label for the new instance
- Count how many of these points are within a defined `distance_threshold` of the new instance (`same_label_points`)
- Compute reliability:
  - reliability=  `same_label_points` / `minimum_cluster_units`
    - `minimum_cluster_units` <- param 
    - If the `same_label_points` size is larger than the threshold (minimum units): reliability = 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.neighbors import NearestNeighbors
from utils.reliability_visualizer import ReliabilityVisualizer

###########################################################
# Load Classifier and Data
###########################################################
pipeline = joblib.load('pipeline_classifier.pkl')
training_data = pd.read_csv("./data_train.csv")

X_train = training_data.drop(columns=['SOURCE'])
y_train = training_data['SOURCE'].map({'out': 0, 'in': 1})
X_train_preprocessed = pipeline.named_steps['preprocessor'].transform(X_train)

In [ ]:
###########################################################
# Reliability Calculation (Density - Distance: Euclidean)
###########################################################
def reliability_distance_based(new_instance, X_train_preprocessed, y_train, predicted_label, distance_threshold=0.10, target_instances=20):
    """
    Calculate the reliability based on euclidean distance and min. number of neighbors

    Reliability = 
        if (neighbors_new_instance < target_instances) 
            neighbors_new_instance / target_instances  
        else
            1

    Parameters:
    - new_instance: Preprocessed new instance.
    - X_train_preprocessed: Preprocessed training data.
    - y_train: Training labels.
    - predicted_label: Predicted class label of the new instance.
    - distance_threshold: The distance threshold for considering neighbors.
    - target_instances: Number of instances for full reliability.

    Returns:
    - reliability_score: [0; 1] reliability.
    """
    distances = np.linalg.norm(X_train_preprocessed - new_instance, axis=1)
    
    same_class_neighbors = (y_train == predicted_label)
    close_neighbors = np.sum((distances <= distance_threshold) & same_class_neighbors)
    
    reliability_score = min(close_neighbors / target_instances, 1.0)
    return reliability_score

In [ ]:
###########################################################
# Classification and Reliability
###########################################################
def classify_and_check_reliability(new_data, pipeline, X_train_preprocessed, y_train, distance_threshold=0.10, target_instances=20):
    """
    Predicts the class and assesses reliability of a new instance.

    Parameters:
    - new_data (pd.DataFrame): DataFrame with the same columns as training data.
    - pipeline (Pipeline): The loaded pipeline for preprocessing and classification.
    - X_train_preprocessed: Preprocessed training data.
    - y_train: Training labels.
    - distance_threshold: The distance threshold for considering neighbors.
    - target_instances: The number of instances required for full reliability.

    Returns:
    - dict: Contains prediction and reliability score.
    """

    # Preprocess & Predict
    new_data_preprocessed = pipeline.named_steps['preprocessor'].transform(new_data)    
    predicted_label = pipeline.named_steps['classifier'].predict(new_data_preprocessed)[0]
    
    # Calculate reliability
    reliability_score = reliability_distance_based(
        new_data_preprocessed[0],
        X_train_preprocessed,
        y_train,
        predicted_label,
        distance_threshold,
        target_instances
    )
    
    return {
        "predicted_label": predicted_label,
        "reliability_score": reliability_score
    }

In [ ]:
###########################################################
#  Load and Test with Unseen Data
###########################################################
validation_data = pd.read_csv("./data_validation.csv")
validation_data['SOURCE'] = validation_data['SOURCE'].map({'out': 0, 'in': 1})
results = []

##############
### Adjust

distance_threshold = 0.10723016075771301
target_instances = 9

for _, instance in validation_data.iterrows():
    true_label = instance['SOURCE']
    instance_df = pd.DataFrame([instance.drop(labels=['SOURCE'])])
    result = classify_and_check_reliability(
        instance_df,
        pipeline,
        X_train_preprocessed,
        y_train,
        distance_threshold=distance_threshold,
        target_instances=target_instances
    )
    result['true_label'] = true_label
    results.append(result)

In [ ]:
###########################################################
# Results
###########################################################
results_df = pd.DataFrame(results)

visualizer = ReliabilityVisualizer(results_df=results_df)
visualizer.plot_reliability_scores((20, 7))

### 10

In [ ]:
reliability_table_10_df = visualizer.get_reliability_table_10()
print(reliability_table_10_df)

In [ ]:
visualizer.plot_table_graph(reliability_table_10_df)

In [ ]:
visualizer.plot_population_and_error_rate_by_class(reliability_table_10_df)

In [ ]:
visualizer.plot_tsne_with_reliability(
    X_train_preprocessed, validation_data, y_train, pipeline
)